In [1]:
# 特征共产鞥

In [4]:
# 导入工具包
from scipy import stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import gc
import copy

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline


# 加载数据
train_data_path = r'E:/DataSet/Tianchi/repeatPurchase/data_format1/train_format1.csv'
test_data_path = r'E:/DataSet/Tianchi/repeatPurchase/data_format1/test_format1.csv'
user_info_path = r'E:/DataSet/Tianchi/repeatPurchase/data_format1/user_info_format1.csv'
user_log_path = r'E:/DataSet/Tianchi/repeatPurchase/data_format1/user_log_format1.csv'

train_data = pd.read_csv(train_data_path, sep=',', encoding='utf-8')
test_data = pd.read_csv(test_data_path, sep=',', encoding='utf-8')
user_info = pd.read_csv(user_info_path, sep=',', encoding='utf-8')
user_log = pd.read_csv(user_log_path, sep=',', encoding='utf-8')

In [5]:
## 定义统计函数
def cnt_(x):
    try:
        return len(x.split(' '))
    except:
        return -1

# 数据唯一值的总数
def nunique_(x):
    try:
        return len(set(x.split(' ')))
    except:
        return -1
    
    
# 统计数据最大值
def max_(x):
    try:
        return np.max([float(i) for i in x.split(' ')])
    except:
        return -1
    
# 统计数据最小值
def min_(x):
    try:
        return np.min([float(i) for i in x.split(' ')])
    except:
        return -1
    
# 统计数据标准差
def std_(x):
    try:
        return np.std([flaot(i) for i in x.split(' ')])
    except:
        return -1
    
# 统计数据中topN的函数
def most_n(x, n):
    try:
        return Counter(x.split(' ').most_comon(n)[n-1][0])
    except:
        return -1
    
def most_n_cnt(x, n):
    try:
        return Counter(x.split(' ')).most_comon(n)[n-1][1]
    except:
        return -1

In [6]:
# 用户特征统计函数
def user_cnt(df_data, single_col, name):
    df_data[name] = df_data[single_col].apply(cnt_)
    return df_data


def user_nuique(df_data,single_col, name):
    df_data[name] = df_data[single_col].apply(nunique_)
    return df_data

def user_max(df_data, single_col, name):
    df_data[name] = df_data[single_col].apply(max_)
    return df_data

def user_min(df_data, single_col, name):
    df_data[name] = df_data[single_col].apply(min_)
    return df_data

def user_std(df_data, single_col, name):
    df_data[name] = df_data[single_col].apply(std_)
    return df_data

def user_most_n(df_data, single_col, name, n=1):
    func = lambda x: most_n(x, n)
    df_data[name] = df_data[single_col].apply(func)
    return df_data


def user_most_n_cnt(df_data, single_col, name, n=1):
    func = lambda x: most_n_cnt(x, n)
    df_data[name] = df_data[single_col].apply(func)
    return df_data


In [ ]:
# 定义内存压缩方法，（实际上就是用占内存少的数据类型来存储数据）
def reduce_mem_usage(df: pd.DataFrame, verbose=True) -> pd.DataFrame:
    start_mem = df.memory_useage().sum() / 1024 * 2
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    
    for col in df.columns:
        col_type = df[col].dtypes
        if (col_type in numerics):
            col_min = df[col].min()
            col_max = df[col].max()
        if (str(col_type)[:3] == 'int'):
            if (col_min > np.iinfo(np.int8).min and col_max < np.iinfo(np.int8)):
                df[col] = df[col].astype(np.int8)
            elif (col_min > np.iifo(np.int16).min and col_max < np.iinfo(np.int16)):
                df[col] = df[col].astype(np.int16)
            elif (col_min > np.iifo(np.int32).min and col_max < np.iinfo(np.int32)):
                df[col] = df[col].astype(np.int32)
            elif (col_min > np.iifo(np.int64).min and col_max < np.iinfo(np.int16)):
                df[col] = df[col].astype(np.int16)

In [7]:
np.iinfo?